In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from keras import backend as K
from tensorflow import keras
#import keras

import tensorflow.keras.applications

import cv2
from PIL import Image

from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint
#from keras.optimizers import Adam
from tensorflow.keras.optimizers import Adam
#from tensorflow.python.keras.layers import *
from keras.layers import Input, Dense, Flatten, Dropout
#from keras.applications.mobilenetv2 import MobileNetV2

import tensorflow as tf

from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Reshape

import numpy as np
from keras.models import Model
from keras.models import model_from_json
import matplotlib.pyplot as plt
import os

from tensorflow.keras.layers import BatchNormalization

#from keras.models import Model
#from keras.layers import Input, Dense, Flatten, Dropout

In [3]:
#Variable Definition
img_w = 512
img_h = 512
channels = 3
classes = 1
info = 5
grid_w = 16
grid_h = 16

In [4]:
def load_model(strr):        
    json_file = open(strr, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    return loaded_model
    

def yolo_model(input_shape):
    
    
    inp = Input(input_shape)
   
    model = MobileNetV2( input_tensor= inp , include_top=False, weights='imagenet')
    last_layer = model.output
    
    conv = Conv2D(512,(3,3) , activation='relu' , padding='same')(last_layer)
    conv = Dropout(0.4)(conv)
    bn = BatchNormalization()(conv)
    lr = LeakyReLU(alpha=0.1)(bn)
    
    
    conv = Conv2D(128,(3,3) , activation='relu' , padding='same')(lr)
    conv = Dropout(0.4)(conv)
    bn = BatchNormalization()(conv)
    lr = LeakyReLU(alpha=0.1)(bn)
    
    
    conv = Conv2D(5,(3,3) , activation='relu' , padding='same')(lr)
    
    final = Reshape((grid_h,grid_w,classes,info))(conv)
    
    model = Model(inp,final)
    
    return model

In [5]:
input_size = (512, 512, 3)

model = yolo_model(input_size)

print(model.summary())

9420800/9406464 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 256, 256, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 256, 256, 32  128         ['Conv1[0][0]']                  
                               

In [6]:
def predict_func(model , inp , iou , name):

    ans = model.predict(inp)
    
    #np.save('Results/ans.npy',ans)
    boxes = decode(ans[0] , img_w , img_h , iou)
    
    img = ((inp + 1)/2)
    img = img[0]
    #plt.imshow(img)
    #plt.show()


    for i in boxes:

        i = [int(x) for x in i]

        img = cv2.rectangle(img , (i[0] ,i[1]) , (i[2] , i[3]) , color = (0,255,0) , thickness = 2)
    
    plt.imshow(img)
    plt.show()
    

    cv2.imwrite(os.path.join('Results/final' , str(name) + '.jpg') , img*255.0)
    return boxes
    
    

In [7]:
def get_biggest_box_coordinate(boxes):
    
    max = 0
    
    for box in boxes:
        bb = (abs(box[0] - box[2])) * (abs(box[1] - box[3]))
        
        if bb >= max:
            max = bb
            biggest_box_coordinate = [box[0], box[1], box[2], box[3]]
    
    return biggest_box_coordinate

In [8]:
def get_biggest_box_image(img, coordinate):
    
    for c in range(4):
        if coordinate[c] < 0 :
            coordinate[c] = 0
    
    xmin = int(coordinate[0])
    xmax = int(coordinate[2])
    ymin = int(coordinate[1])
    ymax = int(coordinate[3])
    
    img_cropped = img[ymin:ymax, xmin:xmax]
    
    return img_cropped

In [10]:
cd /content/drive/MyDrive/tf/

/content/drive/MyDrive/tf


In [ ]:
model = load_model('textline_detect_model2.json')
model.load_weights('textline_detect2.h5')

In [16]:
path = '/content/drive/MyDrive/tf/Test/'
save_path = '/content/drive/MyDrive/tf/Results/'

for i in os.listdir(path):
    img_path = path + str(i)

    img = cv2.imread(os.path.join(img_path), cv2.IMREAD_COLOR)
    
    try:
        img = cv2.resize(img, (512,512), interpolation=cv2.INTER_AREA)
    except:
        break
    img = (img - 127.5)/127.5
    
    boxes = predict_func(model , np.expand_dims(img,axis= 0) , 0.5 , i)

    if len(boxes) == 0:
            biggest_string = img[0:10, 0:10]
    else:
        coor = get_biggest_box_coordinate(boxes)
        biggest_string = get_biggest_box_image(img, coor)

    plt.imshow(biggest_string)
    plt.show()

    cv2.imwrite(os.path.join(save_path , i + '.jpg') , biggest_string*255.0)


NameError: ignored